# Standard Softmax

## Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Dataset

In [2]:
# Load the CIFAR100 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Convert labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)

169001437/169001437 [==============================] - 4s 0us/step


## Model

In [3]:
# Define the model architecture
model = keras.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dense(100, activation="softmax"),
    ]
)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 144s 178ms/step - loss: 3.7353 - accuracy: 0.1350 - val_loss: 3.1932 - val_accuracy: 0.2280
Epoch 2/10
782/782 [==============================] - 134s 171ms/step - loss: 2.9103 - accuracy: 0.2793 - val_loss: 2.7204 - val_accuracy: 0.3233
Epoch 3/10
782/782 [==============================] - 137s 176ms/step - loss: 2.5231 - accuracy: 0.3590 - val_loss: 2.5277 - val_accuracy: 0.3539
Epoch 4/10
782/782 [==============================] - 135s 172ms/step - loss: 2.2481 - accuracy: 0.4162 - val_loss: 2.4319 - val_accuracy: 0.3833
Epoch 5/10
782/782 [==============================] - 133s 170ms/step - loss: 2.0225 - accuracy: 0.4663 - val_loss: 2.3233 - val_accuracy: 0.4087
Epoch 6/10
782/782 [==============================] - 129s 165ms/step - loss: 1.8245 - accuracy: 0.5084 - val_loss: 2.4229 - val_accuracy: 0.4000
Epoch 7/10
782/782 [==============================] - 133s 170ms/step - loss: 1.6435 - accuracy: 0.5512 - val_loss: 2.3812 -

## Evaluation of Model

In [4]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Predict on the test set
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

# Convert one-hot encoding back to labels
y_true = np.argmax(y_test, axis=1)

# Compute evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
confusion = confusion_matrix(y_true, y_pred)

# Print results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion)

313/313 [==============================] - 8s 26ms/step
Accuracy: 0.4152
Precision: 0.4342838332714296
Recall: 0.41519999999999996
F1 Score: 0.41567851811053136
Confusion Matrix:
 [[71  0  0 ...  0  0  0]
 [ 1 49  0 ...  0  0  0]
 [ 1  2 25 ...  1  9  2]
 ...
 [ 0  0  0 ... 33  0  0]
 [ 1  0  4 ...  1 25  1]
 [ 0  0  0 ...  0  2 47]]


# Different Softmax

## Gumbel-Softmax

In [5]:
# Define the Gumbel-Softmax function
def gumbel_softmax(logits, temperature):
    # Sample from a Gumbel distribution
    u = tf.random.uniform(tf.shape(logits), minval=0, maxval=1)
    gumbel = -tf.math.log(-tf.math.log(u + 1e-20) + 1e-20)
    
    # Add the Gumbel noise to the logits and apply temperature
    y = logits + gumbel
    y = y / temperature
    
    # Compute the softmax
    y = tf.nn.softmax(y)
    
    return y


# Define the custom loss function using the Gumbel-Softmax function
def gumbel_softmax_loss(y_true, y_pred):
    y_pred = gumbel_softmax(y_pred, temperature)
    loss = keras.losses.categorical_crossentropy(y_true, y_pred)
    return loss

## Dataset

In [6]:
# Load the CIFAR100 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Convert labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)

## Model

In [7]:
# Define the model architecture
model = keras.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dense(100, activation=None),
    ]
)

# Define the temperature for the Gumbel-Softmax function
temperature = 0.5
# Compile the model with the custom loss function
model.compile(optimizer="adam", loss=gumbel_softmax_loss, metrics=["accuracy"])

# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
782/782 [==============================] - 138s 174ms/step - loss: 8.1812 - accuracy: 0.1134 - val_loss: 7.3058 - val_accuracy: 0.1862
Epoch 2/10
782/782 [==============================] - 141s 180ms/step - loss: 6.6097 - accuracy: 0.2415 - val_loss: 6.2432 - val_accuracy: 0.2715
Epoch 3/10
782/782 [==============================] - 130s 166ms/step - loss: 5.8226 - accuracy: 0.3176 - val_loss: 5.8222 - val_accuracy: 0.3307
Epoch 4/10
782/782 [==============================] - 129s 165ms/step - loss: 5.2898 - accuracy: 0.3677 - val_loss: 5.4696 - val_accuracy: 0.3536
Epoch 5/10
782/782 [==============================] - 134s 171ms/step - loss: 4.8556 - accuracy: 0.4093 - val_loss: 5.3970 - val_accuracy: 0.3689
Epoch 6/10
782/782 [==============================] - 133s 170ms/step - loss: 4.5082 - accuracy: 0.4474 - val_loss: 5.1709 - val_accuracy: 0.3908
Epoch 7/10
782/782 [==============================] - 133s 170ms/step - loss: 4.2134 - accuracy: 0.4816 - val_loss: 5.0980 -

## Evaluation of Model

In [8]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Evaluate the model on the test set
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Compute the evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="macro")
recall = recall_score(y_true, y_pred, average="macro")
f1 = f1_score(y_true, y_pred, average="macro")
cm = confusion_matrix(y_true, y_pred)

# Print the evaluation metrics
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 score: {:.4f}".format(f1))
print("Confusion matrix:")
print(cm)

313/313 [==============================] - 7s 22ms/step
Accuracy: 0.4152
Precision: 0.4465
Recall: 0.4152
F1 score: 0.4164
Confusion matrix:
[[57  1  0 ...  0  0  0]
 [ 0 45  0 ...  0  0  2]
 [ 0  0 13 ...  2 14  0]
 ...
 [ 0  0  1 ... 49  1  0]
 [ 0  0  4 ...  1 32  0]
 [ 0  0  0 ...  0  0 46]]


# Bonus

In [14]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the input shape
input_shape = (32, 32, 3)

# Define the number of classes
num_classes = 100

# Define the number of attention heads
num_heads = 8

# Define the transformer layer
def transformer_layer(inputs, hidden_size, num_heads):
    # Multi-Head Attention
    attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=hidden_size // num_heads)(inputs, inputs)
    attn_output = layers.Dropout(0.1)(attn_output)
    attn_output = layers.LayerNormalization(epsilon=1e-6)(inputs + attn_output)

    # Feed Forward network
    ffn = tf.keras.Sequential([
        layers.Dense(hidden_size, activation='relu'),
        layers.Dense(hidden_size)
    ])
    ffn_output = ffn(attn_output)
    ffn_output = layers.Dropout(0.1)(ffn_output)
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(attn_output + ffn_output)

    return ffn_output

# Define the transformer-based model
def transformer_model(num_classes, num_heads, hidden_size):
    # Define the inputs
    inputs = layers.Input(shape=input_shape)

    # Preprocessing layers
    x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
    x = layers.experimental.preprocessing.RandomCrop(30, 30)(x)

    # Convolutional layers
    x = layers.Conv2D(32, 3, activation='relu')(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Flatten()(x)

    # Transformer layers
    x = layers.Dense(hidden_size)(x)
    x = layers.Reshape((1, hidden_size))(x)
    x = transformer_layer(x, hidden_size, num_heads)
    x = layers.GlobalAveragePooling1D()(x)

    # Output layer with different softmax functions
    outputs_softmax = layers.Dense(num_classes, activation='softmax')(x)
    outputs_gumbel_softmax = layers.Dense(num_classes, activation='linear')(x)
    outputs_gumbel_softmax = tf.nn.softmax(tf.random.gumbel(tf.shape(outputs_gumbel_softmax)) + outputs_gumbel_softmax)

    # Define the model
    model = tf.keras.Model(inputs=inputs, outputs=[outputs_softmax, outputs_gumbel_softmax])

    return model

# Create an instance of the transformer-based model
model = transformer_model(num_classes=num_classes, num_heads=num_heads, hidden_size=128)

# Compile the model
model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], 
              optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, [y_train, y_train], 
                    batch_size=128, epochs=10, validation_data=(x_test, [y_test, y_test]))


AttributeError: ignored